In [1]:
import numpy as np
import open3d as o3d
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import eigsh


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Method 1: Determining Curvature from Point Cloud Distribution using normals from neighboring points

In [10]:
# --- Load & clean ---
pcd = o3d.io.read_point_cloud("./point_cloud_data/shoe/shoe-poisson-sampled.ply")
pcd = pcd.voxel_down_sample(voxel_size=0.005)
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
    radius=0.02, max_nn=50))
X = np.asarray(pcd.points)

In [11]:

# --- Local PCA curvature: "surface variation" (λ_min / (λ1+λ2+λ3)) ---
nn = 30
nbrs = NearestNeighbors(n_neighbors=nn, algorithm='kd_tree').fit(X)
idx = nbrs.kneighbors(return_distance=False)
curv = np.empty(len(X))
for i, N in enumerate(idx):
    P = X[N] - X[i]
    C = (P.T @ P) / (len(N))         # local covariance
    w = np.linalg.eigvalsh(C)        # λ1<=λ2<=λ3
    curv[i] = w[0] / (w.sum() + 1e-12)

In [12]:

# --- Optional: graph Laplacian embedding (diffusion/HKS-ish) ---
# kNN graph with heat weights
k = 12
nbrs = NearestNeighbors(n_neighbors=k, algorithm='kd_tree').fit(X)
dist, inds = nbrs.kneighbors(X)
sigma2 = np.median(dist[:,1:]**2)    # scale
rows, cols, vals = [], [], []
for i in range(len(X)):
    for d,j in zip(dist[i,1:], inds[i,1:]):      # skip self
        w = np.exp(-d*d / (sigma2+1e-12))
        rows += [i, j]; cols += [j, i]; vals += [w, w]
W = csr_matrix((vals,(rows,cols)), shape=(len(X),len(X)))
D = diags(np.ravel(W.sum(axis=1)))
L = D - W                                     # unnormalized Laplacian
# eigen-embeddings (lowest nontrivial eigenvectors)
m = 8
evals, evecs = eigsh(L.asfptype(), k=m+1, M=D.asfptype(), sigma=0.0)  # generalized eig: L u = λ D u
spec = evecs[:,1:m+1]                           # drop trivial component

In [13]:
# --- Feature stack & clustering ---
# features: curvature, normal orientation, spectral coords
N = np.asarray(pcd.normals)
feat = np.c_[curv, N, spec]
feat = (feat - feat.mean(0)) / (feat.std(0)+1e-12)

k_segments = 3
labels = KMeans(n_clusters=k_segments, n_init=10, random_state=0).fit_predict(feat)

In [14]:
# --- Visualize ---
# simple random color map
colors = np.random.RandomState(42).rand(k_segments,3)
pcd.colors = o3d.utility.Vector3dVector(colors[labels])
o3d.visualization.draw_geometries([pcd])

## Using Mesh

In [ ]:
import numpy as np
import open3d as o3d
import igl
from sklearn.cluster import KMeans

# --- Load points & reconstruct a mesh (Poisson) ---
pcd = o3d.io.read_point_cloud("./point_cloud_data/shoe/shoe-poisson-sampled.ply")
pcd = pcd.voxel_down_sample(0.003)
pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=50))

mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd, depth=10)                       # tune depth for fidelity vs smoothness
mesh = mesh.remove_unreferenced_vertices()
mesh = mesh.filter_smooth_taubin(number_of_iterations=10)
V = np.asarray(mesh.vertices); F = np.asarray(mesh.triangles)

# --- Principal curvatures (DDG) ---
k1, k2, d1, d2 = igl.principal_curvature(V, F)   # principal curvatures & dirs
H = 0.5*(k1 + k2)
K = k1 * k2

# --- Laplace–Beltrami spectra & HKS ---
L = igl.cotmatrix(V, F)                 # sparse
A = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_VORONOI)
# generalized eigs: L φ = λ A φ  (smallest λ)
evals, evecs = igl.eigs(L, A, 50)       # 50 eigenpairs (complex -> take real)
evals = np.real(evals); evecs = np.real(evecs)

def hks(evals, evecs, ts=(1e-4,1e-3,1e-2,1e-1)):
    # classic HKS: sum_k exp(-λ_k t) φ_k^2
    phi2 = evecs**2
    return np.stack([ (np.exp(-evals*t) @ phi2.T) for t in ts ], axis=1)

HKS = hks(evals, evecs)                 # (n_vertices, len(ts))

# --- Cluster on DDG features ---
feat = np.c_[H, K, HKS]
feat = (feat - feat.mean(0)) / (feat.std(0)+1e-12)
labels = KMeans(n_clusters=6, n_init=10, random_state=0).fit_predict(feat)

# --- Colorize & view ---
colors = np.random.RandomState(0).rand(labels.max()+1,3)
mesh.vertex_colors = o3d.utility.Vector3dVector(colors[labels])
o3d.visualization.draw_geometries([mesh])

ValueError: too many values to unpack (expected 4)